In [0]:
%pip install transformers==3.5.1

Python interpreter will be restarted.
Collecting transformers==3.5.1
 Using cached transformers-3.5.1-py3-none-any.whl (1.3 MB)
Collecting filelock
 Using cached filelock-3.0.12-py3-none-any.whl (7.6 kB)
Requirement already satisfied: packaging in /local_disk0/.ephemeral_nfs/envs/pythonEnv-56908ec4-9695-4c14-bae3-16d2a4e42ea7/lib/python3.8/site-packages (from transformers==3.5.1) (20.4)
Requirement already satisfied: requests in /local_disk0/.ephemeral_nfs/envs/pythonEnv-56908ec4-9695-4c14-bae3-16d2a4e42ea7/lib/python3.8/site-packages (from transformers==3.5.1) (2.24.0)
Requirement already satisfied: numpy in /local_disk0/.ephemeral_nfs/envs/pythonEnv-56908ec4-9695-4c14-bae3-16d2a4e42ea7/lib/python3.8/site-packages (from transformers==3.5.1) (1.19.2)
Requirement already satisfied: tqdm>=4.27 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-56908ec4-9695-4c14-bae3-16d2a4e42ea7/lib/python3.8/site-packages (from transformers==3.5.1) (4.50.2)
Collecting sentencepiece==0.1.91
 Using cached sentencepiece-0.1.91-cp38-cp38-manylinux1_x86_64.whl (1.1 MB)
Requirement already satisfied: regex!=2019.12.17 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-56908ec4-9695-4c14-bae3-16d2a4e42ea7/lib/python3.8/site-packages (from transformers==3.5.1) (2020.10.15)
Requirement already satisfied: protobuf in /local_disk0/.ephemeral_nfs/envs/pythonEnv-56908ec4-9695-4c14-bae3-16d2a4e42ea7/lib/python3.8/site-packages (from transformers==3.5.1) (3.13.0)
Processing /root/.cache/pip/wheels/d6/17/75/f2ed13c472c4cecc14f003401bb45efadca64cc589d4bf3103/sacremoses-0.0.44-py3-none-any.whl
Collecting tokenizers==0.9.3
 Using cached tokenizers-0.9.3-cp38-cp38-manylinux1_x86_64.whl (2.9 MB)
Requirement already satisfied: six in /local_disk0/.ephemeral_nfs/envs/pythonEnv-56908ec4-9695-4c14-bae3-16d2a4e42ea7/lib/python3.8/site-packages (from packaging->transformers==3.5.1) (1.15.0)
Requirement already satisfied: pyparsing>=2.0.2 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-56908ec4-9695-4c14-bae3-16d2a4e42ea7/lib/python3.8/site-packages (from packaging->transformers==3.5.1) (2.4.7)
Requirement already satisfied: idna<3,>=2.5 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-56908ec4-9695-4c14-bae3-16d2a4e42ea7/lib/python3.8/site-packages (from requests->transformers==3.5.1) (2.10)
Requirement already satisfied: certifi>=2017.4.17 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-56908ec4-9695-4c14-bae3-16d2a4e42ea7/lib/python3.8/site-packages (from requests->transformers==3.5.1) (2020.12.5)
Requirement already satisfied: chardet<4,>=3.0.2 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-56908ec4-9695-4c14-bae3-16d2a4e42ea7/lib/python3.8/site-packages (from requests->transformers==3.5.1) (3.0.4)
Requirement already satisfied: urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-56908ec4-9695-4c14-bae3-16d2a4e42ea7/lib/python3.8/site-packages (from requests->transformers==3.5.1) (1.25.11)
Requirement already satisfied: setuptools in /local_disk0/.ephemeral_nfs/envs/pythonEnv-56908ec4-9695-4c14-bae3-16d2a4e42ea7/lib/python3.8/site-packages (from protobuf->transformers==3.5.1) (50.3.1.post20201107)
Requirement already satisfied: joblib in /local_disk0/.ephemeral_nfs/envs/pythonEnv-56908ec4-9695-4c14-bae3-16d2a4e42ea7/lib/python3.8/site-packages (from sacremoses->transformers==3.5.1) (0.17.0)
Requirement already satisfied: click in /local_disk0/.ephemeral_nfs/envs/pythonEnv-56908ec4-9695-4c14-bae3-16d2a4e42ea7/lib/python3.8/site-packages (from sacremoses->transformers==3.5.1) (7.1.2)
Installing collected packages: filelock, sentencepiece, sacremoses, tokenizers, transformers
Successfully installed filelock-3.0.12 sacremoses-0.0.44 sentencepiece-0.1.91 tokenizers-0.9.3 transformers-3.5.1
Python interpreter will be restarted.

In [0]:
import glob
#import logging
import os
import pickle
import random
import re
import shutil
from typing import Dict, List, Tuple

import numpy as np
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm, trange 

from transformers import (
    MODEL_WITH_LM_HEAD_MAPPING,
    WEIGHTS_NAME,
    AdamW,
    AutoConfig,
    BertTokenizerFast,
    AutoModelForMaskedLM,
    PreTrainedModel,
    PreTrainedTokenizer,
    get_linear_schedule_with_warmup,
)

In [0]:
try:
    from torch.utils.tensorboard import SummaryWriter
except ImportError:
    from tensorboardX import SummaryWriter


#logger = logging.getLogger(__name__)


MODEL_CONFIG_CLASSES = list(MODEL_WITH_LM_HEAD_MAPPING.keys())
MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)

In [0]:
class TextDataset(Dataset):
    def __init__(self, tokenizer: PreTrainedTokenizer, args, file_path: str, block_size=512):
        assert os.path.isfile(file_path)

        block_size = block_size - tokenizer.num_special_tokens_to_add(pair=False)

        directory, filename = os.path.split(file_path)
        cached_features_file = os.path.join(
            directory, args.model_type + "_cached_lm_" + str(block_size) + "_" + filename
        )

        if os.path.exists(cached_features_file) and not args.overwrite_cache:
            print("Loading features from cached file %s", cached_features_file)
            with open(cached_features_file, "rb") as handle:
                self.examples = pickle.load(handle)
        else:
            print("Creating features from dataset file at %s", directory)
            pass

            self.examples = []
            with open(file_path, encoding="utf-8") as f:
                text = f.read()

            tokenized_text = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text))

            for i in range(0, len(tokenized_text) - block_size + 1, block_size):  # Truncate in block of block_size
                self.examples.append(tokenizer.build_inputs_with_special_tokens(tokenized_text[i : i + block_size]))
            # Note that we are loosing the last truncated example here for the sake of simplicity (no padding)
            # If your dataset is small, first you should loook for a bigger one :-) and second you
            # can change this behavior by adding (model specific) padding.

            print("Saving features into cached file %s", cached_features_file)
            with open(cached_features_file, "wb") as handle:
                pickle.dump(self.examples, handle, protocol=pickle.HIGHEST_PROTOCOL)

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, item):
        return torch.tensor(self.examples[item], dtype=torch.long)


In [0]:
class LineByLineTextDataset_old(Dataset):
    def __init__(self, tokenizer: PreTrainedTokenizer, args, file_path: str, block_size=512):
        assert os.path.isfile(file_path)
        # Here, we do not cache the features, operating under the assumption
        # that we will soon use fast multithreaded tokenizers from the
        # `tokenizers` repo everywhere =)
        print("Creating features from dataset file at %s", file_path)

        with open(file_path, encoding="utf-8") as f:
            lines = [line for line in f.read().splitlines() if (len(line) > 0 and not line.isspace())]

        self.examples = tokenizer.batch_encode_plus(lines, add_special_tokens=True, max_length=block_size)["input_ids"]

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        return torch.tensor(self.examples[i], dtype=torch.long)

In [0]:
class LineByLineTextDataset(Dataset):
    def __init__(self, tokenizer: PreTrainedTokenizer, args, file_path: str, block_size=512):
        assert os.path.isfile(file_path)
        # Here, we do not cache the features, operating under the assumption
        # that we will soon use fast multithreaded tokenizers from the
        # `tokenizers` repo everywhere =)
        print("Creating features from dataset file at %s", file_path)
        self.tokenizer = tokenizer
        self.block_size = block_size
        with open(file_path, encoding="utf-8") as f:
            self.lines = [line for line in f.read().splitlines() if (len(line) > 0 and not line.isspace())]

        #self.examples = tokenizer.batch_encode_plus(lines, add_special_tokens=True, max_length=block_size)["input_ids"]

    def __len__(self):
        return len(self.lines)

    def __getitem__(self, i):
        return torch.tensor(self.tokenizer.batch_encode_plus([self.lines[i]], add_special_tokens=True, max_length=self.block_size)["input_ids"][0], dtype=torch.long)

In [0]:
import pickle
class LineByLineTextDataset_new(Dataset):
    def __init__(self, tokenizer: PreTrainedTokenizer, args, file_path: str, block_size=512):
        assert os.path.isfile(file_path)
        # Here, we do not cache the features, operating under the assumption
        # that we will soon use fast multithreaded tokenizers from the
        # `tokenizers` repo everywhere =)
        print("Creating features from dataset file at %s", file_path)
        self.file_range_list = []
        self.step = 1000000
        
        with open(file_path, encoding="utf-8") as f:
            self.lines = [line for line in f.read().splitlines() if (len(line) > 0 and not line.isspace())]
        
        for i in range(0,self.lines,step):
            examples = tokenizer.batch_encode_plus(lines[i:i+step], add_special_tokens=True, max_length=block_size)["input_ids"]
            self.file_range_list.append((i,i+step))
            with open("/dbfs/mnt/els-nlp-experts1/data/Gizem/examples_"+str(i)+_+str(i+step),"wb") as f:
              pickle.dump(examples,f)

    def __len__(self):
        return len(self.lines)

    def __getitem__(self, i):
        start = None
        for item in self.file_range_list:
          if i>= item[0] and i<item[1]:
            start=item[0]
        with open("/dbfs/mnt/els-nlp-experts1/data/Gizem/examples_"+str(start)+_+str(start+step),"rb") as f:
          examples = pickle.load(f)
        relative_i = i - start
        
        return torch.tensor(examples[relative_i], dtype=torch.long)

In [0]:
def load_and_cache_examples(args, tokenizer, evaluate=False,f=''):
    #file_path = args.eval_data_file if evaluate else args.train_data_file
    file_path = f if evaluate else args.train_data_file
    if args.line_by_line:
        return LineByLineTextDataset(tokenizer, args, file_path=file_path, block_size=args.block_size)
    else:
        return TextDataset(tokenizer, args, file_path=file_path, block_size=args.block_size)


def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)


def _sorted_checkpoints(args, checkpoint_prefix="checkpoint", use_mtime=False) -> List[str]:
    ordering_and_checkpoint_path = []

    glob_checkpoints = glob.glob(os.path.join(args.output_dir, "{}-*".format(checkpoint_prefix)))

    for path in glob_checkpoints:
        if use_mtime:
            ordering_and_checkpoint_path.append((os.path.getmtime(path), path))
        else:
            regex_match = re.match(".*{}-([0-9]+)".format(checkpoint_prefix), path)
            if regex_match and regex_match.groups():
                ordering_and_checkpoint_path.append((int(regex_match.groups()[0]), path))

    checkpoints_sorted = sorted(ordering_and_checkpoint_path)
    checkpoints_sorted = [checkpoint[1] for checkpoint in checkpoints_sorted]
    return checkpoints_sorted


def _rotate_checkpoints(args, checkpoint_prefix="checkpoint", use_mtime=False) -> None:
    if not args.save_total_limit:
        return
    if args.save_total_limit <= 0:
        return

    # Check if we should delete older checkpoint(s)
    checkpoints_sorted = _sorted_checkpoints(args, checkpoint_prefix, use_mtime)
    if len(checkpoints_sorted) <= args.save_total_limit:
        return

    number_of_checkpoints_to_delete = max(0, len(checkpoints_sorted) - args.save_total_limit)
    checkpoints_to_be_deleted = checkpoints_sorted[:number_of_checkpoints_to_delete]
    for checkpoint in checkpoints_to_be_deleted:
        print("Deleting older checkpoint [{}] due to args.save_total_limit".format(checkpoint))
        shutil.rmtree(checkpoint)


def mask_tokens(inputs: torch.Tensor, tokenizer: PreTrainedTokenizer, args) -> Tuple[torch.Tensor, torch.Tensor]:
    """ Prepare masked tokens inputs/labels for masked language modeling: 80% MASK, 10% random, 10% original. """

    if tokenizer.mask_token is None:
        raise ValueError(
            "This tokenizer does not have a mask token which is necessary for masked language modeling. Remove the --mlm flag if you want to use this tokenizer."
        )

    labels = inputs.clone()
    # We sample a few tokens in each sequence for masked-LM training (with probability args.mlm_probability defaults to 0.15 in Bert/RoBERTa)
    probability_matrix = torch.full(labels.shape, args.mlm_probability)
    special_tokens_mask = [
        tokenizer.get_special_tokens_mask(val, already_has_special_tokens=True) for val in labels.tolist()
    ]
    probability_matrix.masked_fill_(torch.tensor(special_tokens_mask, dtype=torch.bool), value=0.0)
    if tokenizer._pad_token is not None:
        padding_mask = labels.eq(tokenizer.pad_token_id)
        probability_matrix.masked_fill_(padding_mask, value=0.0)
    masked_indices = torch.bernoulli(probability_matrix).bool()
    labels[~masked_indices] = -100  # We only compute loss on masked tokens

    # 80% of the time, we replace masked input tokens with tokenizer.mask_token ([MASK])
    indices_replaced = torch.bernoulli(torch.full(labels.shape, 0.8)).bool() & masked_indices
    inputs[indices_replaced] = tokenizer.convert_tokens_to_ids(tokenizer.mask_token)

    # 10% of the time, we replace masked input tokens with random word
    indices_random = torch.bernoulli(torch.full(labels.shape, 0.5)).bool() & masked_indices & ~indices_replaced
    random_words = torch.randint(len(tokenizer), labels.shape, dtype=torch.long)
    inputs[indices_random] = random_words[indices_random]

    # The rest of the time (10% of the time) we keep the masked input tokens unchanged
    return inputs, labels


def train(args, train_dataset, model: PreTrainedModel, tokenizer: PreTrainedTokenizer) -> Tuple[int, float]:
    

    """ Train the model """
    #if args.local_rank in [-1, 0]:
    #    tb_writer = SummaryWriter()

    args.train_batch_size = args.per_gpu_train_batch_size * max(1, args.n_gpu)

    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    train_sampler = RandomSampler(train_dataset) if args.local_rank == -1 else DistributedSampler(train_dataset)
    train_dataloader = DataLoader(
        train_dataset, sampler=train_sampler, batch_size=args.train_batch_size, collate_fn=collate
    )

    if args.max_steps > 0:
        t_total = args.max_steps
        args.num_train_epochs = args.max_steps // (len(train_dataloader) // args.gradient_accumulation_steps) + 1
    else:
        t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs

    model = model.module if hasattr(model, "module") else model  # Take care of distributed/parallel training
    model.resize_token_embeddings(len(tokenizer))

    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": args.weight_decay,
        },
        {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total
    )

    # Check if saved optimizer or scheduler states exist
    if (
        args.model_name_or_path
        and os.path.isfile(os.path.join(args.model_name_or_path, "optimizer.pt"))
        and os.path.isfile(os.path.join(args.model_name_or_path, "scheduler.pt"))
    ):
        # Load in optimizer and scheduler states
        optimizer.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "optimizer.pt")))
        scheduler.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "scheduler.pt")))

    if args.fp16:
        try:
            from apex import amp
        except ImportError:
            raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
        model, optimizer = amp.initialize(model, optimizer, opt_level=args.fp16_opt_level)

    # multi-gpu training (should be after apex fp16 initialization)
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Distributed training (should be after apex fp16 initialization)
    if args.local_rank != -1:
        model = torch.nn.parallel.DistributedDataParallel(
            model, device_ids=[args.local_rank], output_device=args.local_rank, find_unused_parameters=True
        )

    # Train!
    print("***** Running training *****")
    print("  Num examples = %d", len(train_dataset))
    print("  Num Epochs = %d", args.num_train_epochs)
    print("  Instantaneous batch size per GPU = %d", args.per_gpu_train_batch_size)
    print(
       "  Total train batch size (w. parallel, distributed & accumulation) = %d",
        args.train_batch_size
        * args.gradient_accumulation_steps
        * (torch.distributed.get_world_size() if args.local_rank != -1 else 1),
    )
    print("  Gradient Accumulation steps = %d", args.gradient_accumulation_steps)
    print("  Total optimization steps = %d", t_total)
    
  
      
    global_step = 0
    epochs_trained = 0
    steps_trained_in_current_epoch = 0
    # Check if continuing training from a checkpoint
    if args.model_name_or_path and os.path.exists(args.model_name_or_path):
        try:
            # set global_step to gobal_step of last saved checkpoint from model path
            checkpoint_suffix = args.model_name_or_path.split("-")[-1].split("/")[0]
            global_step = int(checkpoint_suffix)
            epochs_trained = global_step // (len(train_dataloader) // args.gradient_accumulation_steps)
            steps_trained_in_current_epoch = global_step % (len(train_dataloader) // args.gradient_accumulation_steps)

            print("  Continuing training from checkpoint, will skip to saved global_step")
            print("  Continuing training from epoch %d", epochs_trained)
            print("  Continuing training from global step %d", global_step)
            print("  Will skip the first %d steps in the first epoch", steps_trained_in_current_epoch)
        except ValueError:
            print("  Starting fine-tuning.")

    tr_loss, logging_loss = 0.0, 0.0

    model.zero_grad()
    train_iterator = trange(
        epochs_trained, int(args.num_train_epochs), desc="Epoch", disable=args.local_rank not in [-1, 0]
    )
    set_seed(args)  # Added here for reproducibility
    for epoch in train_iterator:
        #epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=args.local_rank not in [-1, 0])
        epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=True)

        if args.local_rank != -1:
            train_sampler.set_epoch(epoch)

        for step, batch in enumerate(epoch_iterator):
            # Skip past any already trained steps if resuming training
            if steps_trained_in_current_epoch > 0:
                steps_trained_in_current_epoch -= 1
                continue

            inputs, labels = mask_tokens(batch, tokenizer, args) if args.mlm else (batch, batch)
            inputs = inputs.to(args.device)
            labels = labels.to(args.device)
            model.train()
            outputs = model(inputs, masked_lm_labels=labels) if args.mlm else model(inputs, labels=labels)
            loss = outputs[0]  # model outputs are always tuple in transformers (see doc)

            if args.n_gpu > 1:
                loss = loss.mean()  # mean() to average on multi-gpu parallel training
            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps

            if args.fp16:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
            else:
                loss.backward()

            tr_loss += loss.item()
            if (step + 1) % args.gradient_accumulation_steps == 0:
                if args.fp16:
                    torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args.max_grad_norm)
                else:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
                optimizer.step()
                scheduler.step()  # Update learning rate schedule
                model.zero_grad()
                global_step += 1

                if args.local_rank in [-1, 0] and args.logging_steps > 0 and global_step % args.logging_steps == 0:
                    # Log metrics
                    if (
                        args.local_rank == -1 and args.evaluate_during_training
                    ):  # Only evaluate when single GPU otherwise metrics may not average well
                        results = evaluate(args, model, tokenizer,f='/dbfs/mnt/els-nlp-experts1/data/Gizem/monitor.txt')
                        for key, value in results.items():
                            #tb_writer.add_scalar("eval_{}".format(key), value, global_step)
                            print("eval_"+str(key)+" "+str(value)+" "+str(global_step))
                    #tb_writer.add_scalar("lr", scheduler.get_lr()[0], global_step)
                    #tb_writer.add_scalar("loss", (tr_loss - logging_loss) / args.logging_steps, global_step)
                    print("lr "+ str(scheduler.get_lr()[0])+" "+str(global_step))
                    print("loss "+str( (tr_loss - logging_loss) / args.logging_steps)+" "+str( global_step))
                    logging_loss = tr_loss

                if args.local_rank in [-1, 0] and args.save_steps > 0 and global_step % args.save_steps == 0:
                    checkpoint_prefix = "checkpoint"
                    # Save model checkpoint
                    output_dir = os.path.join(args.output_dir, "{}-{}".format(checkpoint_prefix, global_step))
                    os.makedirs(output_dir, exist_ok=True)
                    model_to_save = (
                        model.module if hasattr(model, "module") else model
                    )  # Take care of distributed/parallel training
                    model_to_save.save_pretrained(output_dir)
                    tokenizer.save_pretrained(output_dir)

                    torch.save(args, os.path.join(output_dir, "training_args.bin"))
                    print("Saving model checkpoint to %s", output_dir)

                    _rotate_checkpoints(args, checkpoint_prefix)

                    torch.save(optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
                    torch.save(scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))
                    print("Saving optimizer and scheduler states to %s", output_dir)

            if args.max_steps > 0 and global_step > args.max_steps:
                epoch_iterator.close()
                break
        if args.max_steps > 0 and global_step > args.max_steps:
            train_iterator.close()
            break

    #if args.local_rank in [-1, 0]:
        #tb_writer.close()

    return global_step, tr_loss / global_step


def evaluate(args, model: PreTrainedModel, tokenizer: PreTrainedTokenizer, prefix="",f='') -> Dict:
    # Loop to handle MNLI double evaluation (matched, mis-matched)
    eval_output_dir = args.output_dir
    eval_dataset = load_and_cache_examples(args, tokenizer, evaluate=True,f=f)

    if args.local_rank in [-1, 0]:
        os.makedirs(eval_output_dir, exist_ok=True)

    args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)
    # Note that DistributedSampler samples randomly

    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(
        eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size, collate_fn=collate
    )

    # multi-gpu evaluate
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Eval!
    #print("***** Running evaluation {} *****".format(prefix))
    #print("  Num examples = %d", len(eval_dataset))
    #print("  Batch size = %d", args.eval_batch_size)
    eval_loss = 0.0
    nb_eval_steps = 0
    model.eval()

    for batch in tqdm(eval_dataloader, desc="Evaluating",disable=True):
        inputs, labels = mask_tokens(batch, tokenizer, args) if args.mlm else (batch, batch)
        inputs = inputs.to(args.device)
        labels = labels.to(args.device)

        with torch.no_grad():
            outputs = model(inputs, masked_lm_labels=labels) if args.mlm else model(inputs, labels=labels)
            lm_loss = outputs[0]
            eval_loss += lm_loss.mean().item()
        nb_eval_steps += 1

    eval_loss = eval_loss / nb_eval_steps
    perplexity = torch.exp(torch.tensor(eval_loss))

    result = {"perplexity": perplexity}

    output_eval_file = os.path.join(eval_output_dir, prefix, "eval_results.txt")
    
    print("Perplexity",perplexity.item())
            
    return result

In [0]:
class arg_class:
    def __init__(self):
        #Input file
        self.train_data_file = "/dbfs/mnt/els-nlp-experts1/data/Gizem/sentences_new.txt"
        #The output directory where the model predictions and checkpoints will be written.
        self.output_dir = "/dbfs/mnt/els-nlp-experts1/data/Gizem/bert-base-cased-tapt3"
        #The model architecture to be trained or fine-tuned.
        self.model_type = "bert-base-cased"
        #An optional input evaluation data file to evaluate the perplexity on (a text file).
        self.eval_data_file = "/dbfs/mnt/els-nlp-experts1/data/Gizem/validation.txt"
        self.line_by_line = True
        self.tokenizer_name = "bert-base-cased"
        self.model_name_or_path ="bert-base-cased"
        self.mlm = True
        self.do_eval = True
        self.do_train = True
        self.evaluate_during_training = True
        self.seed = 0
        
        #??
        self.num_train_epochs = 2
        self.learning_rate = 0.0005
        self.per_gpu_train_batch_size = 4
        self.per_gpu_eval_batch_size = 16
        self.gradient_accumulation_steps = 2048/self.per_gpu_train_batch_size #or 256/self.per_gpu_train_batch_size
        self.logging_steps = 20 #Change this to be at the end of each epoch
        self.save_steps = 20 #Change this to be at the end of each epoch
        #??
        
        #I won't touch these
        self.warmup_steps = 0
        self.weight_decay = 0.0
        self.adam_epsilon = 1e-8
        self.max_grad_norm = 1.0
        self.server_ip = None
        self.server_port = None
        self.mlm_probability = 0.15
        self.should_continue = False
        self.config_name = None
        self.cache_dir = None
        self.block_size = -1
        self.max_steps = -1
        self.save_total_limit = None
        self.eval_all_checkpoints = False
        self.no_cuda = False
        self.overwrite_output_dir = False
        self.overwrite_cache = False
        self.fp16 = False
        self.fp16_opt_level = "O1"
        self.local_rank = -1

In [0]:
args = arg_class()

if (
    os.path.exists(args.output_dir)
    and os.listdir(args.output_dir)
    and args.do_train
    and not args.overwrite_output_dir
    and not args.should_continue
):
    raise ValueError(
        "Output directory ({}) already exists and is not empty. Use --overwrite_output_dir to overcome.".format(
            args.output_dir
        )
    )

device = torch.device('cuda')
args.n_gpu=1   
args.device = device

In [0]:
#dbutils.fs.rm("dbfs:/mnt/els-nlp-experts1/data/Gizem/bert-base-cased-tapt3/checkpoint-980/config.json")
#dbutils.fs.rm("dbfs:/mnt/els-nlp-experts1/data/Gizem/bert-base-cased-tapt3/checkpoint-980/optimizer.pt")
#dbutils.fs.rm("dbfs:/mnt/els-nlp-experts1/data/Gizem/bert-base-cased-tapt3/checkpoint-980/pytorch_model.bin")
#dbutils.fs.rm("dbfs:/mnt/els-nlp-experts1/data/Gizem/bert-base-cased-tapt3/checkpoint-980/scheduler.pt")
#dbutils.fs.rm("dbfs:/mnt/els-nlp-experts1/data/Gizem/bert-base-cased-tapt3/checkpoint-980/special_tokens_map.json")
#dbutils.fs.rm("dbfs:/mnt/els-nlp-experts1/data/Gizem/bert-base-cased-tapt3/checkpoint-980/tokenizer_config.json")
#dbutils.fs.rm("dbfs:/mnt/els-nlp-experts1/data/Gizem/bert-base-cased-tapt3/checkpoint-980/training_args.bin")
#dbutils.fs.rm("dbfs:/mnt/els-nlp-experts1/data/Gizem/bert-base-cased-tapt3/checkpoint-980/vocab.txt")
#dbutils.fs.rm("dbfs:/mnt/els-nlp-experts1/data/Gizem/bert-base-cased-tapt3/checkpoint-980/")

Out[49]: True

In [0]:
#dbutils.fs.cp("dbfs:/mnt/els-nlp-experts1/data/Gizem/bert-base-cased-tapt3","dbfs:/FileStore/shared_uploads/g.aydxn@elsevier.com/",recurse=True)

Out[52]: True

In [0]:
#dbutils.fs.rm("dbfs:/mnt/els-nlp-experts1/data/Gizem/bert-base-cased-tapt/eval_results.txt")
#dbutils.fs.ls("dbfs:/mnt/els-nlp-experts1/data/Gizem/bert-base-cased-tapt/")

In [0]:
# Setup logging
#logging.basicConfig(
#    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
#    datefmt="%m/%d/%Y %H:%M:%S",
#    level=logging.INFO if args.local_rank in [-1, 0] else logging.WARN,
#)
#logger.warning(
#    "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
#    args.local_rank,
#    device,
#    args.n_gpu,
#    bool(args.local_rank != -1),
#    args.fp16,
#)

In [0]:
# Set seed
set_seed(args)

In [0]:
config = AutoConfig.from_pretrained("bert-base-cased")
tokenizer = BertTokenizerFast.from_pretrained("bert-base-cased")

In [0]:
model = AutoModelForMaskedLM.from_pretrained(
        args.model_name_or_path,
        from_tf=bool(".ckpt" in args.model_name_or_path),
        config=config,
        cache_dir=args.cache_dir,
        )    
args.block_size = tokenizer.max_len
model.to(args.device)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/local_disk0/.ephemeral_nfs/envs/pythonEnv-56908ec4-9695-4c14-bae3-16d2a4e42ea7/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1421: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
 warnings.warn(
Out[12]: BertForMaskedLM(
 (bert): BertModel(
 (embeddings): BertEmbeddings(
 (word_embeddings): Embedding(28996, 768, padding_idx=0)
 (position_embeddings): Embedding(512, 768)
 (token_type_embeddings): Embedding(2, 768)
 (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
 (dropout): Dropout(p=0.1, inplace=False)
 )
 (encoder): BertEncoder(
 (layer): ModuleList(
 (0): BertLayer(
 (attention): BertAttention(
 (self): BertSelfAttention(
 (query): Linear(in_features=768, out_features=768, bias=True)
 (key): Linear(in_features=768, out_features=768, bias=True)
 (value): Linear(in_features=768, out_features=768, bias=True)
 (dropout): Dropout(p=0.1, inplace=False)
 )
 (output): BertSelfOutput(
 (dense): Linear(in_features=768, out_features=768, bias=True)
 (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
 (dropout): Dropout(p=0.1, inplace=False)
 )
 )
 (intermediate): BertIntermediate(
 (dense): Linear(in_features=768, out_features=3072, bias=True)
 )
 (output): BertOutput(
 (dense): Linear(in_features=3072, out_features=768, bias=True)
 (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
 (dropout): Dropout(p=0.1, inplace=False)
 )
 )
 (1): BertLayer(
 (attention): BertAttention(
 (self): BertSelfAttention(
 (query): Linear(in_features=768, out_features=768, bias=True)
 (key): Linear(in_features=768, out_features=768, bias=True)
 (value): Linear(in_features=768, out_features=768, bias=True)
 (dropout): Dropout(p=0.1, inplace=False)
 )
 (output): BertSelfOutput(
 (dense): Linear(in_features=768, out_features=768, bias=True)
 (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
 (dropout): Dropout(p=0.1, inplace=False)
 )
 )
 (intermediate): BertIntermediate(
 (dense): Linear(in_features=768, out_features=3072, bias=True)
 )
 (output): BertOutput(
 (dense): Linear(in_features=3072, out_features=768, bias=True)
 (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
 (dropout): Dropout(p=0.1, inplace=False)
 )
 )
 (2): BertLayer(
 (attention): BertAttention(
 (self): BertSelfAttention(
 (query): Linear(in_features=768, out_features=768, bias=True)
 (key): Linear(in_features=768, out_features=768, bias=True)
 (value): Linear(in_features=768, out_features=768, bias=True)
 (dropout): Dropout(p=0.1, inplace=False)
 )
 (output): BertSelfOutput(
 (dense): Linear(in_features=768, out_features=768, bias=True)
 (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
 (dropout): Dropout(p=0.1, inplace=False)
 )
 )
 (intermediate): BertIntermediate(
 (dense): Linear(in_features=768, out_features=3072, bias=True)
 )
 (output): BertOutput(
 (dense): Linear(in_features=3072, out_features=768, bias=True)
 (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
 (dropout): Dropout(p=0.1, inplace=False)
 )
 )
 (3): BertLayer(
 (attention): BertAttention(
 (self): BertSelfAttention(
 (query): Linear(in_features=768, out_features=768, bias=True)
 (key): Linear(in_features=768, out_features=768, bias=True)
 (value): Linear(in_features=768, ou

In [0]:
result = evaluate(args, model, tokenizer,f='/dbfs/mnt/els-nlp-experts1/data/Gizem/validation.txt')
result = evaluate(args, model, tokenizer,f='/dbfs/mnt/els-nlp-experts1/data/Gizem/monitor.txt')

Creating features from dataset file at %s /dbfs/mnt/els-nlp-experts1/data/Gizem/validation.txt
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/local_disk0/.ephemeral_nfs/envs/pythonEnv-56908ec4-9695-4c14-bae3-16d2a4e42ea7/lib/python3.8/site-packages/transformers/modeling_bert.py:1150: FutureWarning: The `masked_lm_labels` argument is deprecated and will be removed in a future version, use `labels` instead.
 warnings.warn(
Perplexity 14.76291275024414
Creating features from dataset file at %s /dbfs/mnt/els-nlp-experts1/data/Gizem/monitor.txt
Perplexity 15.499031066894531

In [0]:
# Training
if args.do_train:
    #if args.local_rank not in [-1, 0]:
    #    torch.distributed.barrier()  # Barrier to make sure only the first process in distributed training process the dataset, and the others will use the cache

    train_dataset = load_and_cache_examples(args, tokenizer, evaluate=False)

    if args.local_rank == 0:
        torch.distributed.barrier()

    global_step, tr_loss = train(args, train_dataset, model, tokenizer)
    print(" global_step = %s, average loss = %s", global_step, tr_loss)#13830960

Creating features from dataset file at %s /dbfs/mnt/els-nlp-experts1/data/Gizem/sentences_new.txt
***** Running training *****
 Num examples = %d 13831064
 Num Epochs = %d 2
 Instantaneous batch size per GPU = %d 4
 Total train batch size (w. parallel, distributed & accumulation) = %d 2048.0
 Gradient Accumulation steps = %d 512.0
 Total optimization steps = %d 13506.0
Epoch: 0%| | 0/2 [00:00<?, ?it/s]Creating features from dataset file at %s /dbfs/mnt/els-nlp-experts1/data/Gizem/monitor.txt
Perplexity 6.022308826446533
eval_perplexity tensor(6.0223) 20
/local_disk0/.ephemeral_nfs/envs/pythonEnv-56908ec4-9695-4c14-bae3-16d2a4e42ea7/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:247: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
 warnings.warn("To get the last learning rate computed by the scheduler, "
lr 0.0004992595883311121 20
loss 2.6585879570775433 20
Saving model checkpoint to %s /dbfs/mnt/els-nlp-experts1/data/Gizem/bert-base-cased-tapt3/checkpoint-20
/local_disk0/.ephemeral_nfs/envs/pythonEnv-56908ec4-9695-4c14-bae3-16d2a4e42ea7/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
 warnings.warn(SAVE_STATE_WARNING, UserWarning)
Saving optimizer and scheduler states to %s /dbfs/mnt/els-nlp-experts1/data/Gizem/bert-base-cased-tapt3/checkpoint-20
Creating features from dataset file at %s /dbfs/mnt/els-nlp-experts1/data/Gizem/monitor.txt
Perplexity 5.1747050285339355
eval_perplexity tensor(5.1747) 40
lr 0.0004985191766622241 40
loss 1.9056920712522696 40
Saving model checkpoint to %s /dbfs/mnt/els-nlp-experts1/data/Gizem/bert-base-cased-tapt3/checkpoint-40
Saving optimizer and scheduler states to %s /dbfs/mnt/els-nlp-experts1/data/Gizem/bert-base-cased-tapt3/checkpoint-40
Creating features from dataset file at %s /dbfs/mnt/els-nlp-experts1/data/Gizem/monitor.txt
Perplexity 4.590297698974609
eval_perplexity tensor(4.5903) 60
lr 0.0004977787649933363 60
loss 1.7878313557215733 60
Saving model checkpoint to %s /dbfs/mnt/els-nlp-experts1/data/Gizem/bert-base-cased-tapt3/checkpoint-60
Saving optimizer and scheduler states to %s /dbfs/mnt/els-nlp-experts1/data/Gizem/bert-base-cased-tapt3/checkpoint-60
Creating features from dataset file at %s /dbfs/mnt/els-nlp-experts1/data/Gizem/monitor.txt
Perplexity 4.365617752075195
eval_perplexity tensor(4.3656) 80
lr 0.0004970383533244484 80
loss 1.7272635620684014 80
Saving model checkpoint to %s /dbfs/mnt/els-nlp-experts1/data/Gizem/bert-base-cased-tapt3/checkpoint-80
Saving optimizer and scheduler states to %s /dbfs/mnt/els-nlp-experts1/data/Gizem/bert-base-cased-tapt3/checkpoint-80
Creating features from dataset file at %s /dbfs/mnt/els-nlp-experts1/data/Gizem/monitor.txt
Perplexity 4.116343975067139
eval_perplexity tensor(4.1163) 100
lr 0.0004962979416555605 100
loss 1.679799532686593 100
Saving model checkpoint to %s /dbfs/mnt/els-nlp-experts1/data/Gizem/bert-base-cased-tapt3/checkpoint-100
Saving optimizer and scheduler states to %s /dbfs/mnt/els-nlp-experts1/data/Gizem/bert-base-cased-tapt3/checkpoint-100
Creating features from dataset file at %s /dbfs/mnt/els-nlp-experts1/data/Gizem/monitor.txt
Perplexity 4.124221324920654
eval_perplexity tensor(4.1242) 120
lr 0.0004955575299866726 120
loss 1.6490687134384643 120
Saving model checkpoint to %s /dbfs/mnt/els-nlp-experts1/data/Gizem/bert-base-cased-tapt3/checkpoint-120
Saving optimizer and scheduler states to %s /dbfs/mnt/els-nlp-experts1/data/Gizem/bert-base-cased-tapt3/checkpoint-120
Creating features from dataset file at %s /dbfs/mnt/els-nlp-experts1/data/Gizem/monitor.txt
Perplexity 3.8804993629455566
eval_perplexity tensor(3.8805) 140
lr 0.0004948171183177846 140
loss 1.6205465500352148 140
Saving model checkpoint to %s /dbfs/mnt/els-nlp-experts1/data/Gizem/bert-base-cased-tapt3/checkpoint-140
Saving optimizer and scheduler states to %s /dbfs/mnt/els-nlp-exper